# Experiment 2 - Fine Tuning Analysis Pipeline
*By Andrew Balch*

In [3]:
import cv2 as cv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
%matplotlib inline 

## Background Removal (leave for later?)
*This one may be put on hold since I have no idea how it will impact the spectral information in the image.*

Test different methods of background removal (by subtraction). The original idea was to use ImageJ's sliding paraboloid method and test different radiuses. A github implementation in opencv is available [here](https://github.com/mbalatsko/opencv-rolling-ball). Open-CV has its own built-in methods of background subtraction that could also be tested, but I like the idea of using the sliding paraboloid better since it was developed for biomedical applications. **Keep in mind that we only want background subtraction to impact the area outside of the emission lines.**

In [ ]:
def background_rm():
    return

## Image Transformation
For proper analysis, all images within the same experimental setup generation must have identical regions of interest (ROIs). Meaning, the spectra should be horizontal with the colors going from blue on the right to red on the left. This is easier for Gen 2 images, but all Gen 1 images have an awkward angle that must be accounted for (see below).

Two methods for adjusting for this were considered. Rotation ~90 deg ccw makes emission lines vertical, but the overall spectra will be slanted towards the lower right corner. This means that the region of interest will include the bottom of the blue emission line straight through to the top of the red emission line. Rotation ~105 deg ccw makes the spectra roughly horizontal but the emission lines themselves will have an obtuse angle. With this a straight line can be drawn through the center of all of the emission lines, but profile generation will include the background in the vertical space at a pixel position in the x-axis at the edges of each emission lines. The resulting profiles of a blank sample with an LED bulb source for these two methods were overlaid to demonstrate their differences (see below). As the resulting profile shapes are near identical, we will use 90 deg ccw rotation for simplicity.

|Gen 1 Calibration Image|Gen 2 Calibration Image|
|:-:|:-:|
|![Gen 1](./Spectra/Gen%201/Calibration/IMG_6421.JPG "Gen 1 Calibration Image")|![Gen 2](./Spectra/Gen%202/Calibration/IMG_6561.JPG "Gen 2 Calibration Image")|

Profile comparisons

![Profile comparison](Gen%201%20LED%20Bulb%20Rotation%20Comparison.jpg)

In [ ]:
def transform(image, gen):
    if gen == 1: 
        # 105 deg ccw will make spectra horizontal, 90 deg ccw will make emission lines vertical
        image = cv.rotate(image, cv.ROTATE_90_COUNTERCLOCKWISE)

    elif gen == 2:
        # Rotate 90 deg cw
        image = cv.rotate(image, cv.ROTATE_90_CLOCKWISE)

    else:
        assert "Invalid/no generation specified, check inputs!"

    return image

## Image Profile Generation
Port ImageJ's [Plot Profile](https://imagej.nih.gov/ij/docs/guide/146-30.html#toc-Subsection-30.11) feature as closely as possible. This involves taking a rectangular region through the center of the spectra (the ROI) and averaging the pixel intensity values in the y-axis (the height of the region) for each pixel in the x-axis (the length of the region). In other words, we want an array with the same size as the length of the image (in pixels) where each element is the averaged intensity of a vertical slice at that pixel.

**Every profile in a generation should share the exact same ROI for calibration purposes. Additionally, images should be converted to grayscale before generating the profile to get singular intensity values and avoid dealing with each RGB channel individually. Take care when selecting the method of grayscale conversion.**

Example ImageJ Profile Generation of CFL Spectra in Gen 2 Setup

![Example ImageJ Profile Generation of CFL Spectra in Gen 2 Setup](Profile_Example.jpg)

In [ ]:
def get_profile(image, gen):
    return

## Peak Detection
By-the-book peak detection for any input series (image profile, absorbance, transmission)

In [ ]:
def calc_peaks(profile):
    return

## Calibration to CFL Spectrum
The profile for each image is a dataset of average intensity values for each pixel in the length of the ROI. We need to map these pixel positions to wavelengths to generate absorption and transmission spectra. To do this, we fit a linear equation to pixel positions of known emission peaks of a compact fluorescent light (CFL) ([source](https://commons.wikimedia.org/wiki/File:Fluorescent_lighting_spectrum_peaks_labelled.png)).

Each generation has its own set of calibration images of a blank sample with a CFL source. We'll create the calibration equation by taking the profiles of each calibration image, averaging them, and running peak detection. Visual inspection of the profile plot and these peaks will reveal which pixel positions correspond to the wavelengths at which CFL peaks are known to occur (see the below example). Finding a line of best fit through these pixel position & wavelength points gives us the calibration equation.

![Labeled calibration spectra](Labeled_calibration_spectra.jpg)

In [ ]:
def get_calibration_eqn(gen):
    eqn = None
    if gen == 1: 
        # TODO: Set eqn to manually determined calibration eqn

    elif gen == 2:
        # TODO: Set eqn to manually determined calibration eqn

    else:
        assert "Invalid/no generation specified, check inputs!"

    return eqn

## Moving Averages
After we have mapped pixel positions to wavelengths with the calibration equation we want to experiment with taking a moving average across the series. Basically, we have a window of a specified size (in nm, wavelength units) and we take the average over all of the intensity values at a wavelength in that window, then move to the next. For example, a window of 5 nm would average the intensities within 400 to 405 nm, then those within 405 to 410, etc. 

We want to test 4 window lengths:
1. None 
2. 3 nm
3. 5 nm
4. 10 nm

**Note:** Taking the moving average will result in having a single value represent a range of wavelengths. To get around any issues this may cause, we'll say that the averaged value is at a wavelength in the middle of the window. Ex\) The average in a window from 400 to 405 nm will represent a value at 402.5 nm

In [ ]:
def moving_average(series, window_len=None):
    if window_len is not None:
        return series
    
    return series

## Reference Calculation Using Blanks
Getting transmission and absorption data requires a reference set for maximum light passthrough (blank samples). We are testing three methods for this: 
1. Averaging the profiles for each blank sample
2. Taking the "maximum" profile curve (the single sample with the largest intensity values overall)
3. Taking the argmax of all samples (largest intensity at a wavelength across all blanks)

In [ ]:
def get_reference(method="avg"):
    """Method param can be one of "avg" (default), "maxcurve", or "argmax" """
    return

## Calculate Transmittance
The transmittance of light through the sample is calculated by dividing the intensity of the sample (transmitted light) by the reference value (incident light) at each wavelength.

In [ ]:
def calc_trans(sample, reference):
    return

## Calculate Absorbance
Now that we have transmittance, we can calculate the absorbance of light by the sample by the equation: $absorbance=-log_{10}(transmittance)$.

In [ ]:
def calc_abs(transmittance):
    return

## Driver Code
Putting it all together into a pipeline that takes each image, tests all parameters, and generates graphs and data for inspection. Here is the hierarchy of parameters we want to test: 
* Generation (2 params)
    * Light source (4 params)
        * Background removal (~3 params, if applicable)
            * Reference method (3 params)
                * Moving average (4 params)
                

For each combination of parameters, we want to end up with 3 datasets (plots and raw data): one of the reference, one of all the TJ supplement samples, and another of all the Walgreens samples (288 datasets total, 864 if background removal).

**Notes for implementation:** 
* Calibration eqn only needs to be retrieved once for each generation
* Creating a pandas dataframe could be very helpful for plotting
* Save the data in a file structure that follows the above hierarchy

**Notes for plotting:** 
* Reference plot should be wavelength vs intensity for the method used
* Plots of the supplement samples should include transmittance and absorbance stacked on the same x-axis wavelength. They should also include all tested concentrations as different colored lines with a legend (see below example, ignore concentration)

![Example Plot](example_plot.jpeg)